In [ ]:
import os

In [ ]:
# Check if extracted data folder exists
if(not os.path.exists('SYSC4415W23_A3_dataset')):

  # Download and extract the dataset if the zip file does not exist
  if (not os.path.isfile('SYSC4415W23_A3_dataset.zip')):
    !wget https://github.com/jrgreen7/SYSC4906/releases/download/Assignment3/SYSC4415W23_A3_dataset.zip
    !unzip SYSC4415W23_A3_dataset.zip

In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

train_labels = pd.read_csv('SYSC4415W23_A3_dataset/train/labels.csv')
train_features = pd.read_csv('SYSC4415W23_A3_dataset/train/extracted_features.csv')
print(f'number of original features: {train_features.shape}')

# Remove features that have the same value across the board
sel = VarianceThreshold(threshold=(0.95 * (1 - 0.95)))
new_feat = sel.fit_transform(train_features)
print(f'number of reduced features: {new_feat.shape}')
# features.dtypes

number of original features: (1621, 7048)
number of reduced features: (1621, 5424)


/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_variance_threshold.py:111: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


In [ ]:
from sklearn import svm

svm_model = svm.LinearSVC(kernel='linear')

svm_model.fit()

# Create SVM

